# Trip Planning Agent – Concierge Capstone

## 1. 簡介 (Introduction)

這是一個旅行計畫 Agent（Trip Planning Agent），用來幫助使用者根據出發地、目的地、天數、預算和偏好，自動生成每日行程 (itinerary)，並提供天氣與景點建議。

Key Concepts (本專案將涵蓋)：
- **Tools**：使用天氣 API 和地點 (POI) API  
- **Memory**：使用短期 Session 和長期偏好記憶  
- **Multi-Agent**：Orchestrator Agent + 子 Agent  
- **Agent Evaluation**：設計幾種旅行場景，評估生成行程的合理性  

## 2. 環境設定 (Setup)

安裝必要套件 (假設你用 ADK /某個 agent framework)

In [6]:
!pip install google-adk google-genai


[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
import os

try:
    GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
    GOOGLE_MAPS_API_KEY = os.getenv("GOOGLE_MAP_API_KEY")
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
    os.environ["GOOGLE_MAPS_API_KEY"] = GOOGLE_MAPS_API_KEY
    print("✅ Setup and authentication complete.")
except Exception as e:
    print(
        f"🔑 Authentication Error: Please make sure you have added 'GOOGLE_API_KEY' to your Kaggle secrets. Details: {e}"
    )

✅ Setup and authentication complete.


import 所需模組 (agent, session, memory, tool)

In [8]:
import json
import requests
import subprocess
import time
import uuid

from google.adk.agents import LlmAgent
from google.adk.agents.remote_a2a_agent import (
    RemoteA2aAgent,
    AGENT_CARD_WELL_KNOWN_PATH,
)
from google.genai import types

from google.adk.a2a.utils.agent_to_a2a import to_a2a
from google.adk.models.google_llm import Gemini
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.genai import types

# Hide additional warnings in the notebook
import warnings

warnings.filterwarnings("ignore")

print("✅ ADK components imported successfully.")

✅ ADK components imported successfully.


## 3. 定義工具 (Tools)

天氣 API Tool：

建立一個 function/class 包裝 OpenWeather (或你選的天氣 API)

定義輸入 (地點、日期) 與輸出 (天氣預報)

景點 / POI 查詢 Tool：

包裝地圖 /地點 API (Google Places / Foursquare /類似)

輸入 (城市、興趣類型、時間) → 輸出地點清單 (景點 +分類 +距離)

(可選) 交通 /時間估算 Tool：

如果你要模擬交通時間 (步行 /開車 /大眾交通)，做簡單估算 function

In [9]:
import requests

def weather_lookup(location: str, date: str) -> dict:
    # Geocoding
    geo_url = f"https://geocoding-api.open-meteo.com/v1/search?name={location}&count=1&language=zh"
    geo_resp = requests.get(geo_url).json()
    if 'results' not in geo_resp or len(geo_resp['results']) == 0:
        return {"error": f"找不到城市: {location}"}

    lat = geo_resp['results'][0]['latitude']
    lon = geo_resp['results'][0]['longitude']

    # 天氣預報
    weather_url = (
        f"https://api.open-meteo.com/v1/forecast?"
        f"latitude={lat}&longitude={lon}"
        f"&daily=temperature_2m_max,temperature_2m_min,precipitation_sum"
        f"&timezone=Asia/Taipei"
    )
    weather_resp = requests.get(weather_url).json()

    # 篩選指定日期
    try:
        idx = weather_resp['daily']['time'].index(date)
        result = {
            "location": location,
            "date": date,
            "temperature_max": weather_resp['daily']['temperature_2m_max'][idx],
            "temperature_min": weather_resp['daily']['temperature_2m_min'][idx],
            "precipitation": weather_resp['daily']['precipitation_sum'][idx]
        }
        return result
    except ValueError:
        return {"error": f"找不到日期 {date} 的天氣資料"}


In [10]:
retry_config = types.HttpRetryOptions(
    attempts=5,  # Maximum retry attempts
    exp_base=7,  # Delay multiplier
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504],  # Retry on these HTTP errors  
)

In [ ]:
weather_agent = LlmAgent(
    name="weather_agent",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    instruction="""
你是一個天氣助理。

使用者會提供城市名稱與日期，請使用 weather_lookup 工具查詢天氣。
回覆時請用自然語言，包含：
- 城市
- 日期
- 最高溫
- 最低溫
- 降雨機率
保持簡短且清楚。
""",
    tools=[weather_lookup]
)


# 這裡直接呼叫 Tool 的 func
# user_query = "請告訴我 Taipei 2025-11-18 的天氣"
# result = weather_agent.run(user_query)
# print(result)  
from google.adk.runners import InMemoryRunner

enhanced_runner = InMemoryRunner(agent=weather_agent)
response = await enhanced_runner.run_debug("請告訴我 Taipei 2025-11-18 的天氣.")


App name mismatch detected. The runner is configured with app name "InMemoryRunner", but the root agent was loaded from "D:\projects\trip-planning-agent\.venv\Lib\site-packages\google\adk\agents", which implies app name "agents".



 ### Created new session: debug_session_id

User > 請告訴我 Taipei 2025-11-18 的天氣.


weather_agent > Taipei 在 2025-11-18 的天氣預測：最高溫 20.3 度，最低溫 15.6 度，降雨機率 8.7%。


In [13]:
import requests
from google.adk.tools.function_tool import FunctionTool

def places_text_search(query: str, tool_context=None) -> dict:
    """
    使用 Google Maps Places Text Search 查詢地點
    query: 查詢字串，例如 "Taipei 101"
    """
    url = "https://maps.googleapis.com/maps/api/place/textsearch/json"
    params = {
        "query": query,
        "key": os.environ["GOOGLE_MAPS_API_KEY"],
        "language": "zh-TW",  # 中文  
    }

    resp = requests.get(url, params=params).json()

    if resp.get("status") != "OK":
        return {"summary": f"查詢失敗: {resp.get('status')}"}

    # 取前 5 筆結果
    results = resp.get("results", [])[:5]
    summary_list = []
    for r in results:
        name = r.get("name")
        address = r.get("formatted_address")
        rating = r.get("rating", "無評分")
        summary_list.append(f"{name}，地址: {address}，評分: {rating}")

    summary_text = "\n".join(summary_list)
    return {"summary": f"查詢 '{query}' 結果如下:\n{summary_text}"}

# 包裝成 FunctionTool
places_tool = FunctionTool(func=places_text_search)


In [15]:
maps_agent = LlmAgent(
    name="maps_agent",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    instruction="""
你是一個地圖查詢助理。

使用者會提供地點名稱或關鍵字，請使用 places_text_search 工具查詢地點。
回覆時用自然語言，包含：
- 地點名稱
- 地址
- 評分（如果有）
保持簡短且清楚。
""",
    tools=[places_tool]
)
runner = InMemoryRunner(agent=maps_agent)
response_list = await runner.run_debug("請幫我查 Taipei 101 附近的餐廳")


App name mismatch detected. The runner is configured with app name "InMemoryRunner", but the root agent was loaded from "D:\projects\trip-planning-agent\.venv\Lib\site-packages\google\adk\agents", which implies app name "agents".



 ### Created new session: debug_session_id

User > 請幫我查 Taipei 101 附近的餐廳


maps_agent > SALT&STONE 台北101餐廳，地址: 110台灣臺北市信義區市府路45號4樓，評分: 4.5
隨意鳥地方101景觀餐廳，地址: 110台灣臺北市信義區信義路五段7號85F，評分: 4.1
一蘭 台灣台北別館，地址: 110台灣臺北市信義區松壽路11號B1F，評分: 4.2
鼎泰豐 101店，地址: 110台灣臺北市信義區市府路45號B1，評分: 4.5
A Beach 101&Pizza，地址: 110台灣臺北市信義區信義路五段18號B1，評分: 4.5


##　4. 記憶系統 (Memory)

Session 管理 (短期記憶)

使用 InMemorySessionService（或類似）來儲存對話上下文

每一輪對話：儲存使用者輸入 + Agent 的回應

長期記憶 (Memory Bank)

設計一個簡單資料結構 (如 dictionary / key-value) 儲存使用者偏好 (例如常去國家 /旅行風格)

提取記憶：Agent 問對話時可讀這些偏好來調整建議

In [16]:

from google.adk.sessions import InMemorySessionService
from google.adk.memory import InMemoryMemoryService
from google.adk.tools import load_memory


# 建立 Session 與 Memory 服務 (Memory Bank)
session_service = InMemorySessionService()
memory_service = InMemoryMemoryService()

# 加載 memory 工具（讓 agent 可以讀記憶）
memory_tool = load_memory


## 5. Agent 架構 (Multi‑Agent)

Orchestrator Agent (主代理)

接收使用者請求 (出發地、目的地、預算、天數、偏好)

拆分子任務 (例如：查景點、估天氣)

子 Agent (至少一個)

例如 “景點 Agent”：負責根據城市 +偏好給景點建議

“天氣 Agent”：負責查天氣並回報

Agent 通訊

Orchestrator 呼叫子 Agent (function 呼叫方式即可)

子 Agent 回傳結果給 Orchestrator，用來組裝最終行程

In [17]:
from google.adk.a2a.utils.agent_to_a2a import to_a2a


In [18]:
retry_config = types.HttpRetryOptions(
    attempts=5,  # Maximum retry attempts
    exp_base=7,  # Delay multiplier
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504],  # Retry on these HTTP errors  
)

In [19]:

maps_agent = LlmAgent(
    name="maps_agent",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    instruction="""
你是一個地圖查詢助理。

使用者會提供地點名稱或關鍵字，請使用 places_text_search 工具查詢地點。
回覆時用自然語言，包含：
- 地點名稱
- 地址
- 評分（如果有）
保持簡短且清楚。
""",
    tools=[places_tool]
)


In [21]:
weather_agent = LlmAgent(
    name="weather_agent",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    instruction="""
你是一個天氣助理。

使用者會提供城市名稱與日期，請使用 weather_lookup 工具查詢天氣。
回覆時請用自然語言，包含：
- 城市
- 日期
- 最高溫
- 最低溫
- 降雨機率
保持簡短且清楚。
""",
    tools=[weather_lookup]
)

In [24]:
# Maps Agent A2A
maps_a2a_app = to_a2a(
    maps_agent,
    port=8001  # 這個 Agent 將被部署在本地的 8001 端口
)

# Weather Agent A2A
weather_a2a_app = to_a2a(
    weather_agent,
    port=8002  # 另一個端口
)

In [ ]:
orchestrator = LlmAgent(
    model="gemini-2.0-flash",  # 你可以選你在 Kaggle 上可用的模型
    name="OrchestratorAgent",
    instruction=(
        "你是主控 agent。你的任務是：接收使用者輸入 (出發地、目的地、天數、預算、旅遊偏好)，"
        "拆解成子任務 (查景點、查天氣)，並組裝成每日行程。"
    ),
    tools=[WeatherTool(), POITool(), memory_tool],
)


In [23]:
from google.adk.agents import LlmAgent
from google.adk.models import Gemini

orchestrator = LlmAgent(
    model="gemini-2.0-flash",
    name="OrchestratorAgent",
    instruction="""
你是一個旅遊行程規劃助理。

你可以使用以下兩個工具：
1️⃣ maps_agent: 查詢地點資訊，包括名稱、地址與評分。
2️⃣ weather_agent: 查詢特定城市的天氣預報，包括最高/最低溫與降雨機率。

當使用者提供城市與日期時：
- 使用 maps_agent 查詢景點或餐廳
- 使用 weather_agent 查詢當地天氣
- 整合回覆成簡短、自然的旅遊建議

保持回覆簡潔且清楚。
"""
)


## 6. 對話與規劃邏輯 (Prompt ＋ Task Logic)

為每個 Agent 寫 prompt：

Orchestrator 的 prompt：詢問使用者初始資訊 (出發地、天數、預算)

景點 Agent 的 prompt：如何根據 “偏好 +城市 +天數” 生成景點清單和每日行程

天氣 Agent 的 prompt：如何查詢和解釋天氣資料

規劃邏輯 (itinerary generation)：

Orchestrator 根據景點清單 +天數 +天氣 (子 Agent 結果) 生成每日行程

可以簡化：每一天選 3–4 個景點 +安排時間 (大致預估移動時間)

## 7. 對話模擬 (Conversation Simulation)

在 Notebook 中做幾個模擬對話範例 (User <-> Agent)

範例 1：3 天歐洲城市旅行

範例 2：5 天自然風景 +登山偏好旅遊

展示 Agent 是怎麼拆任務、呼叫 tool、回應使用者、調整行程

In [ ]:
import subprocess
import os


weather_server_process = subprocess.Popen(
    [
        "uvicorn",
        "weather_server:app",  # Module:app format
        "--host",
        "localhost",
        "--port",
        "8001",
    ],
    cwd="./",  # Run from /tmp where the file is
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE,
    env={**os.environ},  # Pass environment variables (including GOOGLE_API_KEY)
)

map_server_process = subprocess.Popen(
    [
        "uvicorn",
        "map_server:app",  # Module:app format
        "--host",
        "localhost",
        "--port",
        "8002",
    ],
    cwd="./",  # Run from /tmp where the file is
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE,
    env={**os.environ},  # Pass environment variables (including GOOGLE_API_KEY)
)




In [45]:

import requests


# Wait for server to start (poll until it responds)
max_attempts = 30
for attempt in range(max_attempts):
    try:
        response = requests.get(
            "http://localhost:8001/.well-known/agent-card.json", timeout=1
        )
        if response.status_code == 200:
            print(f"\n✅ Product Catalog Agent server is running!")
            print(f"   Server URL: http://localhost:8001")
            print(f"   Agent card: http://localhost:8001/.well-known/agent-card.json")
            print(response.json())
            break
    except requests.exceptions.RequestException:
        time.sleep(5)
        print(".", end="", flush=True)
else:
    print("\n⚠️  Server may not be ready yet. Check manually if needed.")


# Wait for server to start (poll until it responds)
max_attempts = 30
for attempt in range(max_attempts):
    try:
        response = requests.get(
            "http://localhost:8001/.well-known/agent-card.json", timeout=1
        )
        if response.status_code == 200:
            print(f"\n✅ Product Catalog Agent server is running!")
            print(f"   Server URL: http://localhost:8002")
            print(f"   Agent card: http://localhost:8002/.well-known/agent-card.json")
            print(response.json())
            break
    except requests.exceptions.RequestException:
        time.sleep(5)
        print(".", end="", flush=True)
else:
    print("\n⚠️  Server may not be ready yet. Check manually if needed.")



✅ Product Catalog Agent server is running!
   Server URL: http://localhost:8001
   Agent card: http://localhost:8001/.well-known/agent-card.json
{'capabilities': {}, 'defaultInputModes': ['text/plain'], 'defaultOutputModes': ['text/plain'], 'description': 'An ADK Agent', 'name': 'weather_agent', 'preferredTransport': 'JSONRPC', 'protocolVersion': '0.3.0', 'skills': [{'description': '\n你是一個天氣助理。\n\n使用者會提供城市名稱與日期，請使用 weather_lookup 工具查詢天氣。\n回覆時請用自然語言，包含：\n- 城市\n- 日期\n- 最高溫\n- 最低溫\n- 降雨機率\n保持簡短且清楚。\n', 'id': 'weather_agent', 'name': 'model', 'tags': ['llm']}, {'description': 'Call self as a function.', 'id': 'weather_agent-weather_lookup', 'name': 'weather_lookup', 'tags': ['llm', 'tools']}], 'supportsAuthenticatedExtendedCard': False, 'url': 'http://localhost:8001', 'version': '0.0.1'}

✅ Product Catalog Agent server is running!
   Server URL: http://localhost:8002
   Agent card: http://localhost:8002/.well-known/agent-card.json
{'capabilities': {}, 'defaultInputModes': ['text/plain'], 

In [48]:
from google.adk.agents.remote_a2a_agent import (
    RemoteA2aAgent,
    AGENT_CARD_WELL_KNOWN_PATH,
)

# Weather Agent (Remote)
remote_weather_agent = RemoteA2aAgent(
    name="weather_agent",
    description="Remote weather forecast agent via A2A.",
    agent_card=f"http://localhost:8001{AGENT_CARD_WELL_KNOWN_PATH}",
)

# Maps Agent (Remote)
remote_maps_agent = RemoteA2aAgent(
    name="maps_agent",
    description="Remote Google Maps-based place lookup agent via A2A.",
    agent_card=f"http://localhost:8002{AGENT_CARD_WELL_KNOWN_PATH}",
)

In [52]:

orchestrator = LlmAgent(
    model=Gemini(model="gemini-2.0-flash", retry_options=retry_config),
    name="trip_planner_agent",
    description="An orchestrator that plans trips using maps and weather remote A2A agents.",
    instruction="""
你是一個旅遊行程規劃助理。

你可以使用以下兩個遠端子 Agent：
1. maps_agent：查詢地點、餐廳、評分、地址等。
2. weather_agent：查詢城市的天氣預報（高低溫、降雨機率）。

流程：
- 若使用者給地點關鍵字，請呼叫 maps_agent。
- 若使用者給城市＋日期，請呼叫 weather_agent。
- 最後整合地點＋天氣，提供合理的旅遊建議。

回覆請保持簡潔、自然。
""",
    sub_agents=[remote_maps_agent, remote_weather_agent],  # ⭐ 正在這裡！
)

In [53]:
from google.adk.runners import InMemoryRunner

runner = InMemoryRunner(agent=orchestrator)

response = await runner.run_debug("11 月 18 日想去台北，有什麼建議？請評估天氣跟附近景點。")


2025-11-18 14:01:14,293 - WARNING - runners.py:266 - App name mismatch detected. The runner is configured with app name "InMemoryRunner", but the root agent was loaded from "D:\projects\trip-planning-agent\.venv\Lib\site-packages\google\adk\agents", which implies app name "agents".



 ### Created new session: debug_session_id

User > 11 月 18 日想去台北，有什麼建議？請評估天氣跟附近景點。


2025-11-18 14:01:14,733 - INFO - google_llm.py:133 - Sending out request, model: gemini-2.0-flash, backend: GoogleLLMVariant.GEMINI_API, stream: False
2025-11-18 14:01:15,880 - INFO - _client.py:1740 - HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.0-flash:generateContent "HTTP/1.1 200 OK"
2025-11-18 14:01:15,884 - INFO - google_llm.py:186 - Response received from the model.
2025-11-18 14:01:15,885 - WARNING - types.py:6334 - Warning: there are non-text parts in the response: ['function_call'], returning concatenated text result from text parts. Check the full candidates.content.parts accessor to get the full model response.
2025-11-18 14:01:16,134 - INFO - _client.py:1740 - HTTP Request: GET http://localhost:8001/.well-known/agent-card.json "HTTP/1.1 200 OK"
2025-11-18 14:01:16,136 - INFO - card_resolver.py:83 - Successfully fetched agent card data from http://localhost:8001/.well-known/agent-card.json: {'capabilities': {}, 'defaultInputModes': ['t

weather_agent > OK. What year are you planning to visit Taipei?


## 8. 評估 (Evaluation)

設計簡單場景 (scenario) 並測試：

不同城市 /天數 /預算 /偏好

看生成的行程是否合理 (景點分佈是否符合天數)

收集指標 (可以自己定義)：

行程完整度 (每天是否有景點)

偏好匹配度 (Agent 建議是否符合使用者偏好)

使用者接受率 (你自己當使用者，是否願意採納建議)

(可選) 用簡單表格在 Notebook 裡顯示比較結果

## 9. 技術說明與核心概念對應 (Write‑up)

用 Markdown 解釋你專案架構：Agent 架構 +記憶 +tool

列出你在這個專案裡用了哪些比賽要求的 key concepts，比如：

Tools (天氣 +景點 API)

Session & Memory (短期 + 長期)

Multi‑Agent (Orchestrator +子 Agents)

Agent Evaluation (你設計的評估)

描述你為什麼這樣設計 (trade‑off /簡化決策)